In [1]:
import pandas as pd 
import numpy as np
import ast
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [2]:
path = r'Classification Codes Data\msc_codes.txt'

In [3]:
with open(path, 'r', encoding='utf8') as f:
    msc = f.readlines()

In [4]:
codes = []
descriptions = []
for entry in msc:
    code, description = entry.split(' ', 1)
    codes.append(code)
    descriptions.append(description.replace('\n', ''))
    assert len(codes) == len(descriptions)

In [84]:
df = pd.DataFrame({'code': codes, 'description': descriptions})

In [85]:
def extract_identifier(code):
    identifier_0 = code[:2]
    identifier_1 = code[2:3]
    identifier_2 = code[3:5]
    return [identifier_0, identifier_1, identifier_2]
df['identifier'] = df['code'].apply(extract_identifier)

In [86]:
def heirarchy(identifier):
    if re.match(r'\b^[0-9][0-9]\b', identifier[0]) and (identifier[1] == '' and identifier[2] == ''):
        return 1
    if re.match(r'\b^[0-9][0-9]\b', identifier[0]) and (identifier[2] == 'XX'):
        return 1
    if (re.match(r'\b^[0-9][0-9]\b', identifier[0]) and re.match(r'\b^[0-9][0-9]\b', identifier[0])) and (identifier[1] == '-'):
        return 2
    if re.match(r'\b^[0-9][0-9]\b', identifier[0]) and re.match(r'\b^[A-Z]\b', identifier[1]) and re.match(r'\b^xx\b', identifier[2]):
        return 3
    else:
        return 4
df['heirarchy'] = df['identifier'].apply(heirarchy).astype(int)

In [87]:
df.head()

,code,description,heirarchy
0,00,General and overarching topics; collections,1
1,01,History and biography,1
2,03,Mathematical logic and foundations,1
3,05,Combinatorics,1
4,45,Integral equations,1
